In [1]:
import os, sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import logging
import time
import pickle as pkl
from input_output import read_csv, write_csv
from utils import get_df_info, get_current_time
from bs4 import BeautifulSoup
import codecs
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import Pipeline
from functools import reduce

logging.getLogger('matplotlib').setLevel(logging.ERROR)

logger = logging.getLogger(__name__)

if not os.path.exists('./logs'):
    os.mkdir('./logs')

In [2]:
current_time = get_current_time()
f_hdlr = logging.FileHandler('./logs/{}.log'.format(current_time))
fmt = '%(asctime)s [%(levelname)s]: %(name)s - %(message)s'
hdlr = [f_hdlr, logging.StreamHandler()]
logging.basicConfig(level=logging.DEBUG, format=fmt, handlers=hdlr)

np.random.seed(42)

path = './data/FINANCIALNEWS_sample.csv'
df = read_csv(path)

df = df.set_index('OBJECT_ID')

print(df.head(10))
get_df_info(df)

2020-05-16 20:10:03,917 [INFO]: input_output - Read 1000 rows and 15 cols from ./data/FINANCIALNEWS_sample.csv.


Current Time = 2020-05-16_20:10:03
                  PUBLISHDATE                                      TITLE  \
OBJECT_ID                                                                  
379746418  2019-4-23 13:26:52                    公募基金二季报重仓股全揭露，8类排名一文打尽！   
453849759  2019-1-25 19:02:01                         民航局：将开展无人机适航审定分级管理   
453856916  2019-1-25 20:42:35                 民航局：推动形成基于运行风险的民用无人机适航管理模式   
453941181  2019-1-28 00:53:00                促进民营经济发展 14地设纾困基金 总规模超1330亿   
453948030  2019-1-28 07:55:16               国资委鼓励外资参与国企改革，业内人士建议做好资本进出安排   
453949829  2019-1-28 08:13:16                   趣墅时隔一年再获千万融资，别墅度假领域战火重燃？   
453954720  2019-1-28 09:06:18                  融资客青睐28股，豫能控股买入占比高达43.91%   
453955303  2019-1-28 09:12:13                    2018年哈萨克斯坦铁合金产量同比增长7.9%   
453956430  2019-1-28 09:22:45  中银国际*每日速递20190128：顺鑫农业；基金四季度持仓报告；计算机行业点评等   
453957310  2019-1-28 09:32:15                            蓝色起源开始建厂生产火箭发动机   

                                                    

In [3]:
def get_text(c):
    soup = BeautifulSoup(c, 'html.parser')
    return soup.getText()

In [4]:
df['clean_text'] = df['CONTENT'].apply(lambda x: get_text(x))

In [5]:
from zhon.hanzi import punctuation
from zhon.hanzi import non_stops
import re
print(punctuation)
#"《" == "《"
df.clean_text.apply(lambda x: re.sub("[\s+\.\!\/_,$%^*(+\"\']+|[+——！？、~@#￥%……&*（）【】《》℃丨“”(a-z)(0-9)(A-Z)：-]", "",x))

＂＃＄％＆＇（）＊＋，－／：；＜＝＞＠［＼］＾＿｀｛｜｝～｟｠｢｣､　、〃〈〉《》「」『』【】〔〕〖〗〘〙〚〛〜〝〞〟〰〾〿–—‘’‛“”„‟…‧﹏﹑﹔·！？｡。


OBJECT_ID
379746418    香港万得通讯社报道，随着月日基金二季报披露的最后节点过去，目前基金重仓股已经全部显露，共涉及...
453849759    据民航局网站日消息，为促进国内民用无人机产业健康成长，月日，民航局印发基于运行风险的无人机适...
453856916    公司讯，月日民航局印发基于运行风险的无人机适航审定指导意见，针对无人机运行场景丰富运行风险多...
453941181    多省市出台措施意见促进民营经济发展明确对暂时困难企业稳贷续贷地设纾困基金总规模超亿自年月日的...
453948030    证券日报报道，近日，国资委会同家中央企业赴瑞士达沃斯出席世界经济论坛年年会有关活动。论坛以全...
453949829    环球旅讯成立于年月的趣墅，是一家通过与地产商及别墅业主合作运营别墅度假村整合热门旅游目的地闲...
453954720    香港万得通讯社报道，月日，数据统计显示，截至收盘，沪深两市共有只个股有融资资金买入。从融资买...
453955303    阿斯塔纳月日消息，哈萨克斯坦统计委员会周三公布的数据显示，年月哈萨克斯坦铁合金产量吨，同比增...
453956430    目录宏观策略策略基金四季度持仓报告重仓股集中度下降，是否是新趋势的开始海外月制造业创近年新低...
453957310    品玩月日报道，蓝色起源已正式开始在美国阿拉巴马州亨茨维尔建厂，为新格伦号和联合发射联盟的火神...
453958250    人工智能的发展已经是不可逆转的趋势，各国都在争相布局这一高新科技产业，中国也不例外。作者|沙...
453958882                                           广发期货?晨会?股指
453959141    证券时报报道，月日，广东省十三届人大二次会议开幕，省长马兴瑞作广东省人民政府工作报告年，我们...
453959732    杭州富阳新兴纸业有限公司杭州新兴纸业有限公司杭州新沃无纺布有限公司名下位于富阳区大源镇新关村...
453960697    证券时报报道，月日，广东省十三届人大二次会议开幕，省长马兴瑞作广东省人民政府工作报告。报告指...
453962765    国企券商多头并进驰援上市公司图文湖北百亿纾困基金蓄势待发楚天都市

In [6]:
df

,PUBLISHDATE,TITLE,CONTENT,SOURCE,URL,SECTIONS,AREACODES,WINDCODES,INDUSTRYCODES,SECTERCODES,KEYWORDS,OPDATE,OPMODE,MKTSENTIMENTS,clean_text
OBJECT_ID,,,,,,,,,,,,,,,
379746418,2019-4-23 13:26:52,公募基金二季报重仓股全揭露，8类排名一文打尽！,"<P style=""MARGIN-BOTTOM: 1.6em; MARGIN-TOP: 0p...",Wind,http://news.windin.com/ns/findsnap.php?sourcet...,人工新闻|基金|标准新闻|旧版新闻|精选新闻|分类,NaN,NaN,NaN,NaN,NaN,2019-4-23 13:32:43,0,NaN,香港万得通讯社报道，随着7月21日基金二季报披露的最后节点过去，目前基金重仓股已经全部显露，...
453849759,2019-1-25 19:02:01,民航局：将开展无人机适航审定分级管理,"<DIV><DIV><P style="";text-align:justify"">据民航局网...",证券时报网,http://kuaixun.stcn.com/2019/0125/14825024.shtml,三板|旧版新闻|精选新闻|分类|宏观政策模型识别|功能关键字|热点专题,NaN,832201.OC:无人机|ON0207:三板股|h11043.CSI:AMAC交运|ON0...,882434.WI:航空货运与物流|882215.WI:航空货运与物流Ⅲ|882104.WI...,NaN,NaN,2019-2-1 11:15:53,0,NaN,据民航局网站25日消息， 为促进国内民用无人机产业健康成长，1月23日，民航局印发《基于运行...
453856916,2019-1-25 20:42:35,民航局：推动形成基于运行风险的民用无人机适航管理模式,"<DIV><DIV><P>e公司讯，1月23日,民航局印发《基于运行风险的无人机适航审定指导...",e公司,http://www.egsea.com/news/detail?id=356159,三板|旧版新闻|精选新闻|分类|宏观政策模型识别|功能关键字|热点专题,NaN,832201.OC:无人机|ON0207:三板股|h11043.CSI:AMAC交运|ON0...,882434.WI:航空货运与物流|882215.WI:航空货运与物流Ⅲ|882104.WI...,NaN,NaN,2019-2-1 11:15:53,0,NaN,"e公司讯，1月23日,民航局印发《基于运行风险的无人机适航审定指导意见》，针对无人机运行场景..."
453941181,2019-1-28 00:53:00,促进民营经济发展 14地设纾困基金 总规模超1330亿,<DIV><DIV><DIV><P><SPAN>多省市出台措施意见促进民营经济发展 明确对暂...,北青网,https://finance.sina.com.cn/roll/2019-01-28/do...,基金|标准新闻|精选新闻|分类|旧版新闻|宏观政策模型识别|功能关键字|热点专题,NaN,000011.SH:上证基金|ON0209:沪深|399103.SZ:乐富|885010.W...,NaN,NaN,NaN,2019-2-2 11:07:30,0,NaN,多省市出台措施意见促进民营经济发展 明确对暂时困难企业稳贷、续贷14地设纾困基金 总规模超1...
453948030,2019-1-28 07:55:16,国资委鼓励外资参与国企改革，业内人士建议做好资本进出安排,"<!--USED_TEMPLATE:无图新闻模板--><P style=""MARGIN-BO...",Wind,http://snap.windin.com/ns/findsnap.php?sourcet...,宏观|国内财经|标准新闻|旧版新闻|功能关键字|直播|分类|人工新闻|精选新闻|热点专题,DQXW01:大中华区|DQXW:地区新闻|DQXW0101:中国|ON05:地区,NaN,NaN,NaN,NaN,2019-2-2 09:32:23,0,NaN,证券日报报道，近日，国资委会同11家中央企业赴瑞士达沃斯出席世界经济论坛2019年年会有关活...
453949829,2019-1-28 08:13:16,趣墅时隔一年再获千万融资，别墅度假领域战火重燃？,"<DIV><DIV><P style=""""><STRONG>【环球旅讯】</strong>成...",环球旅讯,https://www.traveldaily.cn/article/127156,精选新闻|旧版新闻|分类|宏观政策模型识别|功能关键字,NaN,NaN,6225301020:酒店、度假村与豪华游轮|6225301000:酒店、餐馆与休闲Ⅲ|62...,NaN,NaN,2019-2-4 06:18:00,0,NaN,【环球旅讯】成立于2016年1月的趣墅，是一家通过与地产商及别墅业主合作运营别墅度假村、整合...
453954720,2019-1-28 09:06:18,融资客青睐28股，豫能控股买入占比高达43.91%,"<!--USED_TEMPLATE:无图新闻模板--><P style=""MARGIN-BO...",Wind,http://snap.windin.com/ns/findsnap.php?sourcet...,股市|个股|标准新闻|旧版新闻|个股|A股|Wind资讯|功能关键字|直播|精选新闻|自动新...,DQXW:地区新闻|DQXW01:大中华区|DQXW0101:中国|ON05:地区,300113.SZ:顺网科技|002073.SZ:软控股份|000989.SZ:九芝堂|00...,NaN,NaN,NaN,2019-2-1 09:11:39,0,NaN,香港万得通讯社报道，1月25日，Wind数据统计显示，截至收盘，沪深两市共有947只个股有融...
453955303,2019-1-28 09:12:13,2018年哈萨克斯坦铁合金产量同比增长7.9%,<DIV><DIV><DIV><SPAN>\t\t 阿斯塔纳1月16日消息，哈萨克斯坦统计...,兰格钢铁网,http://luliao.lgmi.com/html/201901/28/7251.htm,精选新闻|旧版新闻|分类,NaN,NaN,6215104050:钢铁|6215104000:金属、非金属与采矿|6215100000:...,NaN,NaN,2019-2-4 06:28:05,0,NaN,\t\t 阿斯塔纳1月16日消息，哈萨克斯坦统计委员会周三公布的数据显示，2018年12月...
453956430,2019-1-28 09:22:45,中银国际*每日速递20190128：顺鑫农业；基金四季度持仓报告；计算机行业点评等,"<DIV><DIV><P style=""margin-top:5px;border-radi...",微信,http://mp.weixin.qq.com/s?__biz=MzAxMzAyMzI4NQ...,外汇|标准新闻|外汇|旧版新闻|股票名家|功能关键字|个股|海外中国股|股市|精选新闻|分类,DQXW0101:中国|DQXW01:大中华区|DQXW:地区新闻|DQ0001:中国（临时...,000860.SZ:顺鑫农业|ON0201:A股|EURCNY.EX:欧元中间价|ON023...,NaN,NaN,NaN,2019-2-4 06:34:07,0,NaN,目 录宏观策略【策略】基金四季度持仓报告：重仓股集中度下降，是否是新趋势的开始【海外】1月...


In [7]:
df.WINDCODES

OBJECT_ID
379746418                                                  NaN
453849759    832201.OC:无人机|ON0207:三板股|h11043.CSI:AMAC交运|ON0...
453856916    832201.OC:无人机|ON0207:三板股|h11043.CSI:AMAC交运|ON0...
453941181    000011.SH:上证基金|ON0209:沪深|399103.SZ:乐富|885010.W...
453948030                                                  NaN
453949829                                                  NaN
453954720    300113.SZ:顺网科技|002073.SZ:软控股份|000989.SZ:九芝堂|00...
453955303                                                  NaN
453956430    000860.SZ:顺鑫农业|ON0201:A股|EURCNY.EX:欧元中间价|ON023...
453957310                          744572:BE|3783:公司实体|ON02:公司
453958250                        600794:清华大学|3783:公司实体|ON02:公司
453958882                                                  NaN
453959141                                                  NaN
453959732                  h30049.CSI:AMAC造纸|ON0209:沪深|ON02:公司
453960697                                                  NaN
453962765    ON0201:A股|0218.HK:申万宏源|ON0204:港股

In [8]:
df = df[(~df.clean_text.str.contains('您没有浏览')) & (~df.WINDCODES.isnull())][['PUBLISHDATE','TITLE','WINDCODES','clean_text']]

In [9]:
df['clean_codes'] = df['WINDCODES'].apply(lambda x: x.split('|')).apply(lambda x: [i.split(':')[0] for i in x])

In [10]:
df['n_codes'] = df.clean_codes.apply(lambda x: len(x))

In [11]:
df

,PUBLISHDATE,TITLE,WINDCODES,clean_text,clean_codes,n_codes
OBJECT_ID,,,,,,
453849759,2019-1-25 19:02:01,民航局：将开展无人机适航审定分级管理,832201.OC:无人机|ON0207:三板股|h11043.CSI:AMAC交运|ON0...,据民航局网站25日消息， 为促进国内民用无人机产业健康成长，1月23日，民航局印发《基于运行...,"[832201.OC, ON0207, h11043.CSI, ON0209, ON02]",5
453856916,2019-1-25 20:42:35,民航局：推动形成基于运行风险的民用无人机适航管理模式,832201.OC:无人机|ON0207:三板股|h11043.CSI:AMAC交运|ON0...,"e公司讯，1月23日,民航局印发《基于运行风险的无人机适航审定指导意见》，针对无人机运行场景...","[832201.OC, ON0207, h11043.CSI, ON0209, ON02]",5
453941181,2019-1-28 00:53:00,促进民营经济发展 14地设纾困基金 总规模超1330亿,000011.SH:上证基金|ON0209:沪深|399103.SZ:乐富|885010.W...,多省市出台措施意见促进民营经济发展 明确对暂时困难企业稳贷、续贷14地设纾困基金 总规模超1...,"[000011.SH, ON0209, 399103.SZ, 885010.WI, ON02]",5
453954720,2019-1-28 09:06:18,融资客青睐28股，豫能控股买入占比高达43.91%,300113.SZ:顺网科技|002073.SZ:软控股份|000989.SZ:九芝堂|00...,香港万得通讯社报道，1月25日，Wind数据统计显示，截至收盘，沪深两市共有947只个股有融...,"[300113.SZ, 002073.SZ, 000989.SZ, 002340.SZ, 0...",21
453956430,2019-1-28 09:22:45,中银国际*每日速递20190128：顺鑫农业；基金四季度持仓报告；计算机行业点评等,000860.SZ:顺鑫农业|ON0201:A股|EURCNY.EX:欧元中间价|ON023...,目 录宏观策略【策略】基金四季度持仓报告：重仓股集中度下降，是否是新趋势的开始【海外】1月...,"[000860.SZ, ON0201, EURCNY.EX, ON0236, EURUSD....",8
453957310,2019-1-28 09:32:15,蓝色起源开始建厂生产火箭发动机,744572:BE|3783:公司实体|ON02:公司,PingWest品玩1月28日报道，蓝色起源已正式开始在美国阿拉巴马州亨茨维尔建厂，为新格伦...,"[744572, 3783, ON02]",3
453958250,2019-1-28 09:40:11,人工智能，在中国表现如何？,600794:清华大学|3783:公司实体|ON02:公司,人工智能的发展已经是不可逆转的趋势，各国都在争相布局这一高新科技产业，中国也不例外。作者 |...,"[600794, 3783, ON02]",3
453959732,2019-1-28 09:55:21,富阳新兴纸业有限公司等三企业核心资产司法拍卖公告,h30049.CSI:AMAC造纸|ON0209:沪深|ON02:公司,杭州富阳新兴纸业有限公司、杭州新兴纸业有限公司、杭州新沃无纺布有限公司名下位于富阳区大源镇新...,"[h30049.CSI, ON0209, ON02]",3
453962765,2019-1-28 10:15:18,湖北百亿纾困基金蓄势待发 国企券商多头援上市公司,ON0201:A股|0218.HK:申万宏源|ON0204:港股|H01085.HK:申万宏...,国企、券商多头并进驰援上市公司图文：湖北百亿纾困基金蓄势待发楚天都市报为缓解湖北上市企业阶段...,"[ON0201, 0218.HK, ON0204, H01085.HK, jMhn1VeEU...",7


In [12]:
clean_codes = np.array(reduce(lambda x, y: x+y, df['clean_codes'])).reshape(-1,1)

In [32]:
obj = df.index.repeat(df.n_codes).values.reshape(-1,1)
dates = df.PUBLISHDATE.repeat(df.n_codes).values.reshape(-1,1)

In [33]:
arr = np.hstack([clean_codes, obj, dates])
obj_df = pd.DataFrame(arr, columns=['code', 'obj', 'date'])

In [34]:
good_codes = pd.read_csv('data/allstocks.list', header=None).values.flatten()

In [35]:
obj_df_clean = obj_df[obj_df.code.isin(good_codes)].sort_values('code')

In [38]:
obj_df_clean.date = pd.to_datetime(obj_df_clean.date)

In [50]:
ph = np.zeros(check_time.shape)

In [54]:
obj_df_clean.date.iloc[0].date() == check_time[0]

False

In [92]:
np.argwhere(check_time==pd.to_datetime('2/1/2019'))

array([[17]])

In [159]:
state_date = '1/26/2019'
end_date = '2/15/2019'
check_time = pd.date_range(start=state_date, end=end_date)
n_days = len(check_time)
def test_sum(series):
    a = (series - pd.to_datetime(state_date)).dt.days
    res = np.zeros(n_days)
    res[a] = 1
    return pd.Series(res).ewm(alpha=0.5).mean().tolist()
df_density = obj_df_clean.groupby('code', as_index=False).agg({'date': test_sum})
density_arr = np.array(reduce(lambda x, y: x+y, df_density.date)).reshape(len(df_density), -1)

In [160]:
df_den = pd.DataFrame(density_arr, columns=[str(a.date()) for a in check_time])
df_den.index = df_density.code

In [ ]:
# 查看跟波动率的变化

In [161]:
df_den

,2019-01-26,2019-01-27,2019-01-28,2019-01-29,2019-01-30,2019-01-31,2019-02-01,2019-02-02,2019-02-03,2019-02-04,...,2019-02-06,2019-02-07,2019-02-08,2019-02-09,2019-02-10,2019-02-11,2019-02-12,2019-02-13,2019-02-14,2019-02-15
code,,,,,,,,,,,,,,,,,,,,,
000001.SZ,0.0,0.000000,0.571429,0.266667,0.645161,0.825397,0.409449,0.203922,0.101761,0.050831,...,0.012698,0.006348,0.003174,0.001587,0.000793,0.000397,0.000198,0.000099,0.000050,0.000025
000002.SZ,0.0,0.000000,0.000000,0.000000,0.000000,0.507937,0.755906,0.376471,0.187867,0.093842,...,0.023443,0.011720,0.005860,0.002930,0.001465,0.000732,0.000366,0.000183,0.000092,0.000046
000004.SZ,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.503937,0.250980,0.125245,0.062561,...,0.015629,0.007813,0.003906,0.001953,0.000977,0.000488,0.000244,0.000122,0.000061,0.000031
000009.SZ,0.0,0.000000,0.000000,0.533333,0.258065,0.126984,0.062992,0.031373,0.015656,0.007820,...,0.001954,0.000977,0.000488,0.000244,0.000122,0.000061,0.000031,0.000015,0.000008,0.000004
000018.SZ,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.503937,0.250980,0.125245,0.062561,...,0.015629,0.007813,0.003906,0.001953,0.000977,0.000488,0.000244,0.000122,0.000061,0.000031
000048.SZ,0.0,0.000000,0.571429,0.266667,0.129032,0.571429,0.787402,0.392157,0.195695,0.097752,...,0.024420,0.012209,0.006104,0.003052,0.001526,0.000763,0.000381,0.000191,0.000095,0.000048
000058.SZ,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.503937,0.250980,0.125245,0.062561,...,0.015629,0.007813,0.003906,0.001953,0.000977,0.000488,0.000244,0.000122,0.000061,0.000031
000069.SZ,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.503937,0.250980,0.125245,0.062561,...,0.015629,0.007813,0.003906,0.001953,0.000977,0.000488,0.000244,0.000122,0.000061,0.000031
000088.SZ,0.0,0.000000,0.000000,0.533333,0.258065,0.126984,0.062992,0.031373,0.015656,0.007820,...,0.001954,0.000977,0.000488,0.000244,0.000122,0.000061,0.000031,0.000015,0.000008,0.000004


In [163]:
obj_df_clean.to_csv('code_obj.csv', index=False)

In [164]:
obj_df_clean

,code,obj,date
29,000001.SZ,453954720,2019-01-28 09:06:18
1880,000001.SZ,454309839,2019-01-31 16:32:52
960,000001.SZ,454147929,2019-01-30 02:08:00
4889,000002.SZ,454364141,2019-02-01 07:54:02
2505,000002.SZ,454349437,2019-01-31 23:44:00
2632,000004.SZ,454349850,2019-02-01 01:06:36
535,000009.SZ,454051542,2019-01-29 08:37:47
4292,000018.SZ,454361306,2019-02-01 07:12:51
4584,000048.SZ,454362349,2019-02-01 07:31:32
2515,000048.SZ,454349448,2019-01-31 19:36:00
